In [1]:
import duckdb as db
import pandas as pd
from pathlib import Path
ROOT = Path('../..')
ROOT.resolve()


PosixPath('/Users/lukestrange/Code/housing')

In [2]:
# Connect duckDB
con = db.connect()
# Make an empty dataframe to store all the data
data = []
# Loop through the codes we need
for i in ['E06', 'E07', 'E08', 'E09', 'E10', 'E11', 'E12', 'E47', 'E92']:
    # Read the files from GitHub
    codes = con.sql(f"SELECT * FROM read_json('https://github.com/open-innovations/geography-code-adder/raw/main/docs/data/{i}.json')").df()
    areas = codes.areas.values[0]
    # print(areas)
    # Get areas and descriptions
    # areas = codes.areas.values[0]
    # desc = codes.description.values[0]
    d = pd.DataFrame.from_dict(areas, orient='index')
    # Combine the frames
    # combined = pd.concat([combined, d], axis=0)
    data.append(d)
    
combined = pd.concat(data).drop(columns=[1, 2]) # Not too sure why we're getting these extra cols but they don't have anything in them anyway.
combined.rename(columns={0: 'content'}, inplace=True)

In [3]:
def rename_date(my_dict):
    # Rename the key "date" to "dt"
    if "date" in my_dict:
        my_dict["dt"] = my_dict.pop("date")
    return my_dict

combined['content'] = combined['content'].apply(rename_date)

In [4]:
combined.to_json(ROOT / 'src/data/areas/place-page/_data/areas.json', orient='index', indent=4)